In [35]:
import pandas as pd
import requests
import io

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import plotly.figure_factory as ff
import pickle

In [36]:
df = pd.read_csv('dataset.csv')
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [37]:
print("Demension of the dataset:")
df.shape

Demension of the dataset:


(4424, 35)

In [38]:
df.drop_duplicates(inplace=True)

In [39]:
print("Target : ")
print(df['Target'].unique())

Target : 
['Dropout' 'Graduate' 'Enrolled']


In [40]:
class_mapping = {'Dropout': 0, 'Graduate': 1, 'Enrolled' : 2}
df['Target'] = df['Target'].map(class_mapping)
df

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,0
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,1
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,0
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,1
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,15,1,1,1,1,1,6,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,1
4420,1,1,2,15,1,1,19,1,1,10,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,0
4421,1,1,1,12,1,1,1,22,27,10,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,0
4422,1,1,1,9,1,1,1,22,27,8,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,1


In [41]:
df_ohe = df.copy()
df_ohe.drop(columns=['Marital status'], axis=1, inplace=True)
df_ohe.drop(columns=['Nacionality'], axis=1, inplace=True)
df_ohe.drop(columns=['Educational special needs'], axis=1, inplace=True)
df_ohe.drop(columns=['International'], axis=1, inplace=True)
df_ohe

,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Displaced,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,8,5,2,1,1,13,10,6,10,1,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,0
1,6,1,11,1,1,1,3,4,4,1,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,1
2,1,5,5,1,1,22,27,10,10,1,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,0
3,8,2,15,1,1,23,27,6,4,1,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,1
4,12,1,3,0,1,22,28,10,10,0,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,6,15,1,1,1,1,6,5,0,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,1
4420,1,2,15,1,1,1,1,10,10,1,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,0
4421,1,1,12,1,1,22,27,10,10,1,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,0
4422,1,1,9,1,1,22,27,8,5,1,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,1


In [42]:
cats = ['Marital status',	'Application mode', 'Daytime/evening attendance',	'Previous qualification', 'Nacionality',	"Mother's qualification",	"Father's qualification",	"Mother's occupation", \
         "Father's occupation", 'Debtor', 'Gender',	'Scholarship holder', 'Course', 'International','Tuition fees up to date', 'Educational special needs','Displaced'] 
column_names = df_ohe.columns.tolist()
cats_set = set(cats)
names_set = set(column_names)

cats = names_set.intersection(cats_set)
print("Unique elements:")
for cat in cats:
    print(df_ohe[cat].unique()) 


Unique elements:
[1 0]
[10  4  8 11  6  9  5  2  3 22  7  1 12 39 19 13 29 46 43 34 44 30 41 24
 23 45 35 26 28 36 16 37 31 42 20 15 40 25 21 17 32 38 27 18 14 33]
[ 1 12 16 14  8  3 15  2  4  9 17 11  6  7 13  5 10]
[ 6  4 10  8  5  2 16  1  7  3 12  9 20 28 13 29 23 32 30 18 24 19 11 21
 15 27 31 14 22 17 26 25]
[1 0]
[ 2 11  5 15  3 17 12 10 14 16  6  8 13  9  4  1  7]
[1 0]
[1 0]
[13  1 22 23  3  4 27  2 19 10 25  7  5 24  9 26 18 11 20 21  6  8 17 28
 12 14 16 15 29]
[0 1]
[ 8  6  1 12  9 17 15 16 14  4 13  7  3  2  5 18 10 11]
[0 1]
[10  3 27 28  1 14  5  4 24  2 29  9  7 26 18 30 12 15 25 31 16 11 20 33
 13 32  8  6 21 17 34 23 19 22]


In [43]:
len(cats)

13

In [44]:
print(df.columns[df.isna().any()])
df_ohe = df_ohe.fillna(0)

Index([], dtype='object')


In [45]:
print(df_ohe.shape)
display(df_ohe.head(5))

(4424, 31)


,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Displaced,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,8,5,2,1,1,13,10,6,10,1,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,0
1,6,1,11,1,1,1,3,4,4,1,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,1
2,1,5,5,1,1,22,27,10,10,1,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,0
3,8,2,15,1,1,23,27,6,4,1,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,1
4,12,1,3,0,1,22,28,10,10,0,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,1


In [46]:
df_ohe = pd.get_dummies(df_ohe, columns=cats)
df_ohe


,Application order,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),...,Father's qualification_25,Father's qualification_26,Father's qualification_27,Father's qualification_28,Father's qualification_29,Father's qualification_30,Father's qualification_31,Father's qualification_32,Father's qualification_33,Father's qualification_34
0,5,20,0,0,0,0,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,19,0,6,6,6,14.000000,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,5,19,0,6,0,0,0.000000,0,0,6,...,0,0,1,0,0,0,0,0,0,0
3,2,20,0,6,8,6,13.428571,0,0,6,...,0,0,1,0,0,0,0,0,0,0
4,1,45,0,6,9,5,12.333333,0,0,6,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,6,19,0,6,7,5,13.600000,0,0,6,...,0,0,0,0,0,0,0,0,0,0
4420,2,18,0,6,6,6,12.000000,0,0,6,...,0,0,0,0,0,0,0,0,0,0
4421,1,30,0,7,8,7,14.912500,0,0,8,...,0,0,1,0,0,0,0,0,0,0
4422,1,20,0,5,5,5,13.800000,0,0,5,...,0,0,1,0,0,0,0,0,0,0


In [47]:
column_names = df_ohe.columns.tolist()

cats_set = set(cats)
names_set = set(column_names)

common_strings = names_set.intersection(cats_set)

nats =  [item for item in names_set if item not in common_strings] # numerical attributes
nats.remove('Target')


In [48]:
nats

['Previous qualification_14',
 'Application mode_5',
 'Previous qualification_9',
 'Curricular units 2nd sem (credited)',
 "Father's qualification_17",
 'Previous qualification_5',
 "Father's qualification_33",
 "Mother's occupation_7",
 "Mother's occupation_9",
 'Previous qualification_12',
 "Father's occupation_7",
 'Tuition fees up to date_0',
 'Previous qualification_2',
 'Previous qualification_10',
 "Father's qualification_6",
 'Previous qualification_11',
 "Father's qualification_12",
 "Father's qualification_7",
 "Father's occupation_10",
 'Previous qualification_15',
 'Course_5',
 "Mother's qualification_28",
 'Application mode_7',
 'Application mode_16',
 "Mother's qualification_2",
 "Mother's qualification_9",
 "Father's occupation_41",
 "Father's occupation_25",
 "Mother's qualification_22",
 "Father's occupation_15",
 "Father's qualification_30",
 "Mother's qualification_3",
 "Mother's occupation_30",
 "Mother's occupation_13",
 'Previous qualification_13',
 "Father's occu

In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


numeric_df = df_ohe[nats]

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_df)

scaled_df = pd.DataFrame(scaled_data, columns=nats)

df_ohe[nats] = scaled_df

print("Normalized data:")
print(df_ohe)



Normalized data:
      Application order  Age at enrollment  \
0              0.555556           0.056604   
1              0.111111           0.037736   
2              0.555556           0.037736   
3              0.222222           0.056604   
4              0.111111           0.528302   
...                 ...                ...   
4419           0.666667           0.037736   
4420           0.222222           0.018868   
4421           0.111111           0.245283   
4422           0.111111           0.056604   
4423           0.111111           0.094340   

      Curricular units 1st sem (credited)  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   
...                                   ...   
4419                                  0.0   
4420                                  0.0   
4421                     

In [50]:
df_ohe.drop(df_ohe[df_ohe["Target"]==2].index,inplace=True)

In [61]:
df_ohe.to_csv('OHEdata.csv')

In [20]:
#input data

In [67]:
labs = ['Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation", 'Displaced', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)', 'Unemployment rate',
       'Inflation rate', 'GDP']
cat_labs = ['Application mode', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation", 'Displaced', 'Debtor',
        'Gender', 'Scholarship holder']

In [68]:
arr = [1, 4, 12, 0, 1, 13, 27, 10, 8, 1, 0, 1, 1, 0, 18, 0, 6, 9, 6, 12.2, 0, 0, 6, 6, 6, 13.5, 0, 12.7, 1.4, -1.7]
input_data = pd.DataFrame([arr], columns=labs)

In [69]:
input_df_encoded = pd.get_dummies(input_data, columns=cat_labs)

In [70]:
ohedf = pd.read_csv('OHEdata.csv')
del ohedf[ohedf.columns[0]]
ohedf.columns

Index(['Application order', 'Age at enrollment',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       ...
       'Father's qualification_25', 'Father's qualification_26',
       'Father's qualification_27', 'Father's qualification_28',
       'Father's qualification_29', 'Father's qualification_30',
       'Father's qualification_31', 'Father's qualification_32',
       'Father's qualification_33', 'Father's qualification_34'],
      dtype='object', length=223)

In [71]:
input_df_encoded = input_df_encoded.reindex(columns=ohedf.columns, fill_value=0)

input_df_encoded = input_df_encoded.drop(columns=['Target'], errors='ignore')

In [72]:
input_df_encoded

,Application order,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),...,Father's qualification_25,Father's qualification_26,Father's qualification_27,Father's qualification_28,Father's qualification_29,Father's qualification_30,Father's qualification_31,Father's qualification_32,Father's qualification_33,Father's qualification_34
0,4,18,0,6,9,6,12.2,0,0,6,...,0,0,1,0,0,0,0,0,0,0
